# Current Quarter

<b>01 - Increase total amount of service on the SHN and reliability of that service by 2024</b>

## Routes on the State Highway Network (SHN)

Transit routes along the SHN can be categorized into 3 groups:
1. **On SHN** - where at least 20% of the transit route runs the SHN (within 50 ft) 
2. **Intersects SHN** - where at least 35% of the transit route runs within 0.5 mile of the SHN.
3. **Other** - all other transit routes.

### Metrics
* service hours, service hours per route
* delay hours, delay hours per route
* map of route by category (and by mode)

The metrics are shown for for transit routes **on the SHN** and **intersects SHN**.

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import HTML

import C1_report_metrics as report_metrics
import C2_report_charts as report_charts
from update_vars import BUS_SERVICE_GCS, CURRENT_QUARTER, ANALYSIS_DATE
from shared_utils import portfolio_utils, rt_dates
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis import styleguide

hq_catalog = intake.open_catalog("../high_quality_transit_areas/*.yml")

In [2]:
# parameters cell

In [3]:
%%capture_parameters
QUARTER_CLEANED = CURRENT_QUARTER.replace('_', ' ')
CURRENT_QUARTER, ANALYSIS_DATE, QUARTER_CLEANED

{"CURRENT_QUARTER": "Q4_2023", "ANALYSIS_DATE": "2023-10-11", "QUARTER_CLEANED": "Q4 2023"}


In [4]:
df = report_metrics.prep_data_for_report(ANALYSIS_DATE)

## Statewide Stats for {QUARTER_CLEANED} ({ANALYSIS_DATE})

In [5]:
summary = report_metrics.get_service_hours_summary_table(df)  

In [6]:
all_hours = portfolio_utils.aggregate_by_geography(
    summary.assign(category="All"),
    group_cols = ["category"],
    sum_cols = ["unique_route", "service_hours"]
)

In [7]:
STATEWIDE_HOURS = all_hours.service_hours.iloc[0]
FORMATTED_HOURS = f'{STATEWIDE_HOURS:,}' 
AVG_SERVICE = round(STATEWIDE_HOURS / all_hours.unique_route.iloc[0], 1)

display(
    HTML(
        f"<h3>{FORMATTED_HOURS} total service hours statewide</h3>"
        f"<h3>{AVG_SERVICE} service hours per route statewide</h3>"

    )
)

In [8]:
# Chart utils
HEIGHT = 200
WIDTH = 150

color_dict = {
    "On SHN": cp.CALITP_CATEGORY_BRIGHT_COLORS[3],
    "Intersects SHN": cp.CALITP_CATEGORY_BRIGHT_COLORS[4],
    "Other": cp.CALITP_CATEGORY_BRIGHT_COLORS[1]
}

def make_bar_by_category(df: pd.DataFrame, x_col: str,
                         y_col: str) -> alt.Chart:
    
    base_bar = report_charts.base_bar(df, x_col = x_col)
    legend_sort_order = ["On SHN", "Intersects SHN", "Other"]
    
    y_title = y_col.replace('_', ' ').title()
    
    chart = base_bar.encode(
        x = alt.X(x_col, sort = legend_sort_order),
        y = alt.Y(f"{y_col}:Q", title = f"{y_title}"),
        color = alt.Color(f"{x_col}:N", title = f"{x_col.title()}",
                          scale = alt.Scale(domain = legend_sort_order,
                              range = list(color_dict.values())))
    )
    
    chart = (chart
             .properties(width = WIDTH, height = HEIGHT, 
                         title= f"{y_title} by Category")
            )
    
    return chart

s1 = make_bar_by_category(summary, x_col = "category", 
                          y_col = "service_hours_per_route")
s2 = make_bar_by_category(summary, x_col = "category", 
                          y_col = "service_hours")

combined = alt.hconcat(s1, s2)
combined = styleguide.apply_chart_config(combined)

combined

alt.HConcatChart(...)

In [9]:
service_cols_dict = {
    "category": "Category",
    "service_hours": "Service Hours",
    "pct_service_hours": "% Service Hours",
    "unique_route": "# Routes",
    "pct_unique_route": "% Routes",
    "service_hours_per_route": "Service Hours per Route",
}

summary_styled = portfolio_utils.style_table(
    summary, 
    rename_cols = service_cols_dict, 
    integer_cols = ["Service Hours", "# Routes"],
    one_decimal_cols = ["Service Hours per Route"],
    left_align_cols = "first",
    center_align_cols = "all",
    custom_format_cols = {'{:.1%}': ["% Service Hours", "% Routes"]},
    display_table = True
)

Category,Service Hours,# Routes,% Service Hours,% Routes,Service Hours per Route
On SHN,"12,997",603,14.9%,23.5%,21.6
Intersects SHN,"46,693","1,291",53.5%,50.3%,36.2
Other,"27,550",674,31.6%,26.2%,40.9


## Reliability (Delay)

Note: Not every route has GTFS Real-Time information, which supplies delay data.

**-- Work in progress --**

In [10]:
#delay_df = df[df.merge_delay=="both"]
#delay_summary = report_metrics.get_delay_summary_table(delay_df)

In [11]:
#STATEWIDE_DELAY = delay_summary.delay_hours.sum()
#FORMATTED_HOURS = f'{STATEWIDE_DELAY:,g}' 
#AVG_DELAY = round(STATEWIDE_DELAY / delay_summary.unique_route.sum(), 2)


#display(
#    HTML(
#        f"<h3>{FORMATTED_HOURS} total delay hours statewide</h3>"
#        f"<h3>{AVG_DELAY} delay hours per route statewide</h3>"
#    )
#)

In [12]:
#d1 = make_bar_by_category(delay_summary, x_col = "category", 
#                          y_col = "delay_hours_per_route")
#d2 = make_bar_by_category(delay_summary, x_col = "category", 
#                          y_col = "delay_hours")

#combined = alt.hconcat(d1, d2)
#combined = styleguide.apply_chart_config(combined)

#combined

In [13]:
delay_cols_dict = {
    "category": "Category",
    "delay_hours": "Total Delay Hours",
    "pct_delay_hours": "% Delay Hours",
    "unique_route": "# Routes",
    "pct_unique_route": "% Routes",
    "delay_hours_per_route": "Delay Hours per Route",
}

#delay_summary_styled = portfolio_utils.style_table(
#    delay_summary, 
#    rename_cols = delay_cols_dict, 
#    integer_cols = ["Total Delay Hours", "# Routes"],
#    two_decimal_cols = ["Delay Hours per Route"],
#    left_align_cols = "first",
#    center_align_cols = "all",
#    custom_format_cols = {'{:.1%}': ["% Delay Hours", "% Routes"]},
#    display_table = True
#)

## By District

In [14]:
# Have some rows where district is missing, 
# but only for intersects_shn and other categories
# focus on just the on_shn category and do district breakdown
#df[(df.district.isna())].category.value_counts()

In [15]:
# Chart utils
WIDTH = 300
HEIGHT = 200

by_district_service = report_metrics.by_district_on_shn_breakdown(
    df, ["service_hours", "unique_route"])

bar_total = (report_charts.make_district_bar(
            by_district_service, "service_hours")
            .properties(width=WIDTH, height=HEIGHT)  
            )
bar_avg = (report_charts.make_district_bar(
            by_district_service, "avg_service_hours")
           .properties(width=WIDTH, height=HEIGHT)
          )

service_hours_chart = report_charts.configure_hconcat_charts(
    [bar_avg, bar_total], 
    x_scale="independent", 
    y_scale="independent", 
    chart_title="Service Hours by District")

service_hours_chart

alt.HConcatChart(...)

In [16]:
#by_district_delay = report_metrics.by_district_on_shn_breakdown(
#    df, ["delay_hours", "unique_route"]
#)

#bar_total = (report_charts.make_district_bar(
#            by_district_delay, "delay_hours")
#            .properties(width=WIDTH, height=HEIGHT)      
#            )
#bar_avg = (report_charts.make_district_bar(
#            by_district_delay, "avg_delay_hours")
#           .properties(width=WIDTH, height=HEIGHT)
#          )

#delay_hours_chart = report_charts.configure_hconcat_charts(
#    [bar_avg, bar_total], 
#    x_scale="independent", 
#    y_scale="independent", 
#    chart_title="Delay Hours by District")

#delay_hours_chart

## Map of Routes by Category

In [17]:
def data_for_viz(df: gpd.GeoDataFrame):
    gdf = report_metrics.clean_up_category_values(df)
    
    # line must fall within CA
    ca = hq_catalog.ca_boundary.read().to_crs(f"EPSG: {gdf.crs.to_epsg()}")

    gdf = gpd.sjoin(
        gdf,
        ca,
        how = "inner",
        predicate = "within",
    ).drop(columns= ["index_right"])
    
    # Drop columns that shouldn't get displayed in tooltip
    drop_cols = [
        #"merge_delay", 
        "State", "unique_route"]
    
    gdf2 = gdf.drop(columns = drop_cols)
    
    return gdf2


gdf = data_for_viz(df)

### All Routes

In [ ]:
def make_map(gdf: gpd.GeoDataFrame): 
    # category is going to be sorted alphabetically,
    # so sort our color_dict
    sorted_dict = dict(sorted(color_dict.items()))
    
    m = gdf.explore(
        "category", categorical=True,
        cmap = list(sorted_dict.values()),
        tiles = "Carto DB Positron"
    )

    display(m)
    
make_map(gdf)